In [17]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install tmdbv3api

In [3]:
from tmdbv3api import TMDb
import json
import requests

from tmdbv3api import Movie

In [19]:
class Extract_Movie():
    def __init__(self, tmdb_api_key):  ## initialise with tmdb_api_key
        self.tmdb_movie = Movie()
        self.tmdb = TMDb()
        self.tmdb.api_key = tmdb_api_key

    def get_genre(self,x):
        genres = []
        result = self.tmdb_movie.search(x)
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, self.tmdb.api_key))
        data_json = response.json()
        if data_json['genres']:
            genre_str = " "
            for i in range(0,len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            np.NaN

    def get_director(self,x):
        if " (director)" in x:
            return x.split(" (director)")[0]
        elif " (directors)" in x:
            return x.split(" (directors)")[0]
        else:
            return x.split(" (director/screenplay)")[0]

    def get_actor1(self,x):
        return ((x.split("screenplay); ")[-1]).split(", ")[0])

    def get_actor2(self,x):
        if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
            return np.NaN
        else:
            return ((x.split("screenplay); ")[-1]).split(", ")[1])


    def get_actor3(self,x):
        if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
            return np.NaN
        else:
            return ((x.split("screenplay); ")[-1]).split(", ")[2])

    def get_movies(self, link):

        df1 = pd.read_html(link, header=0)[2]
        df2 = pd.read_html(link, header=0)[3]
        df3 = pd.read_html(link, header=0)[4]
        df4 = pd.read_html(link, header=0)[5]

        df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

        ## get genre
        df['genres'] = df['Title'].map(lambda x: self.get_genre(str(x)))

        df_year = df[['Title','Cast and crew','genres']]

        ## get director
        df_year['director_name'] = df_year['Cast and crew'].map(lambda x: self.get_director(x))

        ## get actor1
        df_year['actor_1_name'] = df_year['Cast and crew'].map(lambda x: self.get_actor1(x))

        ## get actor2
        df_year['actor_2_name'] = df_year['Cast and crew'].map(lambda x: self.get_actor2(x))

        ## get actor3
        df_year['actor_3_name'] = df_year['Cast and crew'].map(lambda x: self.get_actor3(x))

        df_year = df_year.rename(columns={'Title':'movie_title'})

        new_df_year = df_year.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

        new_df_year['actor_2_name'] = new_df_year['actor_2_name'].replace(np.nan, 'unknown')
        new_df_year['actor_3_name'] = new_df_year['actor_3_name'].replace(np.nan, 'unknown')

        new_df_year['movie_title'] = new_df_year['movie_title'].str.lower()

        new_df_year['comb'] = new_df_year['actor_1_name'] + ' ' + new_df_year['actor_2_name'] + ' '+ new_df_year['actor_3_name'] + ' '+ new_df_year['director_name'] +' ' + new_df_year['genres']

        return new_df_year


In [12]:
api_key = 'Your API key'

In [13]:
movie_rec = Extract_Movie(api_key)

In [14]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"

In [18]:
movie_rec.get_movies(link)

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,the strange ones,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,stratton,Dominic Cooper Austin Stowell Gemma Chan Simon...
3,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Western Crime Thriller Drama,sweet country,Bryan Brown Sam Neill unknown Warwick Thornton...
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller Mystery,the commuter,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
...,...,...,...,...,...,...,...
270,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall,Mystery Adventure Comedy Crime,holmes & watson,Will Ferrell John C. Reilly Rebecca Hall Etan ...
271,Adam McKay,Christian Bale,Amy Adams,Steve Carell,Thriller Science Fiction Action Adventure,vice,Christian Bale Amy Adams Steve Carell Adam McK...
272,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux,Drama History,on the basis of sex,Felicity Jones Armie Hammer Justin Theroux Mim...
273,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell,Thriller Crime Drama Action,destroyer,Nicole Kidman Sebastian Stan Toby Kebbell Kary...
